In [2]:
import os
import sys
# sys.path.append('../../')
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import * 
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
user = os.getlogin()
# from inner_libs.libs_doug import * 
# from libs.doug.utils_doug import * 

In [3]:
import duckdb

# Conectar ao DuckDB (ou criar um novo banco de dados em memória)
con = duckdb.connect()

# Ler arquivos Parquet de uma pasta
# Substitua 'caminho/para/sua/pasta/*.parquet' pelo caminho real
path = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\\2025*.parquet"
query = rf'''
    CREATE TABLE dados AS
    SELECT * FROM read_parquet('{path}')
    
   
'''

con.execute(query)


In [7]:

query = f''' 
      WITH tratados AS 
        (
            SELECT *,
            ROW_NUMBER() OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes DESC) AS rn,
            pontos_principalidade - LAG(pontos_principalidade)  OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC) AS var_pontos

            FROM dados
            QUALIFY rn <= 4
            ORDER BY ano_Mes
        ),
        
        marcacoes AS (
            SELECT  
            *,
            CASE WHEN var_pontos < 0 THEN 1 ELSE 0 END AS queda_flag
            FROM tratados 
            WHERE rn <= 3
            
        ),
        checagem AS (
            SELECT *,
            SUM(queda_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_quedas
            FROM marcacoes
        )
        
        SELECT  * FROM checagem WHERE soma_quedas >= 3 and ano_mes = (SELECT MAX(ano_mes) FROM checagem)
        --SELECT  * FROM checagem WHERE cpf_cnpj = '00362741948'
        

        '''
con.sql(query).to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,idade,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_agro_sicredi,total_valor_agro_mercado_scr,total_valor_outros_sicredi,total_valor_agro_mercado_cpr,sow_agro,total_valor_outros_mercado_scr,sow_outros,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_consorcio_motos,possui_adquirencia,possui_consorcio_pesados,possui_consorcio_servicos,possui_consorcio_automoveis,possui_consorcio_imoveis,possui_seguro_rural,possui_previdencia,possui_seguro_residencial,possui_seguro_agricola,possui_seguro_patrimonial,possui_seguro_vida,possui_seguro_automovel,possui_conta_salario,transacao_app,flg_ativo,flg_novo_assoc,flg_idade_maior_18,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,pontos_rural,pontos_planejamento,pontos_protecao,pontos_previdencia,pontos_movimentacao,pontos_cred_cartao,pontos_cred_outros,pontos_investimento,pontos_cred_rural,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,dat_carga_incremental,faixa_categoria,etl_created_on,etl_updated_on,data_insercao,data_atualizacao,filtro_ano_mes_inicial,filtro_ano_mes_final,rn,var_pontos,queda_flag,soma_quedas
0,202506,00385375913,PF,0730,7000,11,121,ATIVO,PF II,71,201907,BAIXO 2,LEGADO,"1,629.75",45,S,S,200.00,S,0.00,"1,829.75","2,217.86",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,"4,040.39","4,020.41",0.00,"8,060.80",20,0,0,0,0,40,0,0,9,0,0,69,80-100%,86.25,2025-07-17,Promissor,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-19,1,3.00
1,202506,00430920393,PF,0730,7000,41,010201,ATIVO,PF II,36,202206,BAIXO 1,DIGITAL,0.00,43,S,S,"6,342.50",S,0.00,"6,342.50","7,846.75","7,846.75",0.00,0.00,824.94,0.00,0.00,"93,565.02",0.01,1.00,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,"9,205.68","10,089.10",0.00,"19,294.78",30,0,0,0,0,60,30,0,31,0,1,151,120+,188.75,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-5,1,3.00
2,202506,00640990916,PF,0730,7000,27,111,ATIVO,PF II,9,202409,BAIXO 2,LEGADO,225.97,42,S,S,"38,525.42",S,0.00,"38,751.39","6,928.82","1,998.10",0.00,0.00,0.00,0.00,0.00,"4,008.33",0.00,0.29,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0.00,393.15,0.00,393.15,15,0,0,0,0,1,8,0,60,0,1,84,100-120%,105.0,2025-07-17,Principal,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-4,1,3.00
3,202506,00674797906,PF,0730,7000,20,131,ATIVO,PF III,100,201702,BAIXÍSSIMO,LEGADO,"3,192.41",45,S,S,"93,321.67",S,0.00,"96,514.08",6.25,6.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,"7,680.00","16,506.27",0.00,"24,186.27",12,0,5,0,0,80,0,0,80,0,1,177,120+,177.0,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-4,1,3.00
4,202506,00766322963,PF,0730,7000,19,131,ATIVO,PF IV,34,202208,BAIXÍSSIMO,LEGADO,761.84,40,S,S,0.00,N,0.00,761.84,"21,465.01","3,748.28",0.00,0.00,0.00,0.00,0.00,0.65,0.00,0.17,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0.00,"1,134.28",0.00,"1,134.28",16,0,0,0,0,3,8,0,0,0,0,27,20-80%,20.76923076923077,2025-07-17,Emergente,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-2,1,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [8]:
df

NameError: name 'df' is not defined

In [ ]:

path_to = criar_pasta_exportacao("Alexandre Hekermann H - leads_visao")
salvar_jupyter_em_pasta(path_to, file_origin="Principal.ipynb", arquivo_reutilizado=False)
base_exportar.to_excel(os.path.join(path_to,"base_pesquisas.xlsx"),index=False)
# total.to_parquet(os.path.join(path_to,"base_total_raw.parquet"),index=False)

# with pd.ExcelWriter(os.path.join(path_to,"lotes_agroleite_2025.xlsx")) as writer:
#     agro_pf_tel.to_excel(writer, sheet_name="Agroleite PF", index=False)
#     agro_pj_tel.to_excel(writer, sheet_name="Agroleite PJ", index=False)

os.startfile(path_to)


['ano_mes', 'cpf_cnpj', 'segmento', 'cod_cooperativa', 'cod_central',
       'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao',
       'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 'des_origem',
       'vlr_capital_social',  'pix_trans_30d', 'cad_pix',
       'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia',
       'sum_investimentos', 'total_valor_cartao_mercado_scr',
       'total_valor_cartao_sicredi', 'total_valor_outros_sicredi',  
         'sow_cartao',
       'possui_cartao_credito', 'debito_conta_ativo', 'possui_domicilio',
       'possui_cartao_debito', 'possui_folha_pagamento', 'possui_cobranca',
       'possui_adquirencia'(maquineta),
         'transacao_app',
       'flg_ativo', 'flg_novo_assoc',  'cad_pix_ativo',
       'ativou_open_finance', 'cash_in', 'cash_out', 'cash_total_fator',
       'cash_total', 'pontos_produtos_basicos',  'flag_principalidade',
       'pontos_principalidade', 'faixa_atingimento_meta',
       'percentual_atingimento',  'faixa_categoria',
        'data_atualizacao',  'var_pontos',
       'queda_flag', 'soma_quedas']